In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
anime = pd.read_csv('anime.csv')
rating = pd.read_csv('rating.csv')
rating.nunique()

user_id     73515
anime_id    11200
rating         11
dtype: int64

In [3]:
rating_count = rating.groupby('anime_id')['rating'].count()
print(rating_count.describe())

count    11200.000000
mean       697.655089
std       2028.627749
min          1.000000
25%          5.000000
50%         51.500000
75%        385.250000
max      39340.000000
Name: rating, dtype: float64


In [4]:
# Remove anime that have been reviewed by less than 50 people
rating_count_sel = rating_count[rating.groupby('anime_id')['rating'].count().values>50].sort_values(ascending=False)
rating_count_sel.index.nunique()

5625

In [5]:
anime_filtered = anime[anime['anime_id'].isin(rating_count_sel.index)].reset_index()

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
anime_filtered['genre'] = anime_filtered['genre'].astype("string")

In [8]:
anime_filtered.to_csv('anime_filtered.csv')

In [9]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')
anime_filtered['genre'] = anime_filtered['genre'].fillna('')

In [10]:
vectorized_data = tfv.fit_transform(anime_filtered['genre'])

In [11]:
genre_tf = pd.DataFrame(vectorized_data.toarray(),columns=tfv.get_feature_names_out())
genre_tf.index = anime_filtered['name']
genre_tf

action  action adventure  \
name                                                           
Kimi no Na wa.                    0.000000          0.000000   
Fullmetal Alchemist: Brotherhood  0.103629          0.158103   
Gintama°                          0.093019          0.000000   
Steins;Gate                       0.000000          0.000000   
Gintama&#039;                     0.093019          0.000000   
...                                    ...               ...   
Pico: My Little Summer Story      0.000000          0.000000   
Pico to Chico                     0.000000          0.000000   
Advancer Tina                     0.000000          0.000000   
Pico x CoCo x Chico               0.000000          0.000000   
Boku no Pico                      0.000000          0.000000   

                                  action adventure cars  \
name                                                      
Kimi no Na wa.                                      0.0   
Fullmetal Alchemist: Brotherhood                    0.0   
Gintama°                                            0.0   
Steins;Gate                                         0.0   
Gintama&#039;                                       0.0   
...                                                 ...   
Pico: My Little Summer Story                        0.0   
Pico to Chico                                       0.0   
Advancer Tina                                       0.0   
Pico x CoCo x Chico                                 0.0   
Boku no Pico                                        0.0   

                                  action adventure comedy  \
name                                                        
Kimi no Na wa.                                        0.0   
Fullmetal Alchemist: Brotherhood                      0.0   
Gintama°                                              0.0   
Steins;Gate                                           0.0   
Gintama&#039;                                         0.0   
...                                                   ...   
Pico: My Little Summer Story                          0.0   
Pico to Chico                                         0.0   
Advancer Tina                                         0.0   
Pico x CoCo x Chico                                   0.0   
Boku no Pico                                          0.0   

                                  action adventure demons  \
name                                                        
Kimi no Na wa.                                        0.0   
Fullmetal Alchemist: Brotherhood                      0.0   
Gintama°                                              0.0   
Steins;Gate                                           0.0   
Gintama&#039;                                         0.0   
...                                                   ...   
Pico: My Little Summer Story                          0.0   
Pico to Chico                                         0.0   
Advancer Tina                                         0.0   
Pico x CoCo x Chico                                   0.0   
Boku no Pico                                          0.0   

                                  action adventure drama  \
name                                                       
Kimi no Na wa.                                  0.000000   
Fullmetal Alchemist: Brotherhood                0.257234   
Gintama°                                        0.000000   
Steins;Gate                                     0.000000   
Gintama&#039;                                   0.000000   
...                                                  ...   
Pico: My Little Summer Story                    0.000000   
Pico to Chico                                   0.000000   
Advancer Tina                                   0.000000   
Pico x CoCo x Chico                             0.000000   
Boku no Pico                                    0.000000   

                                  action adventure ecchi  \
name  

In [12]:
genre_tf.to_csv('genre_tf.csv')

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
anime_data = genre_tf.index
user_input = "Narut"
user_input = user_input.strip().lower()

In [15]:
from fuzzywuzzy import fuzz

# define a function to find the most similar anime title in the dataset based on the user input
def find_similar_anime(user_input, anime_data):
    best_match = None
    best_score = 0
    for anime_title in anime_data:
        score = fuzz.ratio(user_input, anime_title.lower())
        if score > best_score:
            best_score = score
            best_match = anime_title
    return best_match


In [16]:
# find the most similar anime title in the dataset based on the user input
best_match = find_similar_anime(user_input, anime_data)

# print the most similar anime title
print("Most similar anime title:", best_match)

Most similar anime title: Naruto


In [21]:
def recommender(name):
    cos_array = pd.DataFrame(cosine_similarity(genre_tf),columns=anime_filtered['name'],index=anime_filtered['name'])
    recommendation = (cos_array.loc[name].sort_values(ascending=False).index).tolist()
    (recommendation).remove(name)
    return recommendation[:5]

In [22]:
recommender(best_match)

['Naruto x UT',
 'Naruto Soyokazeden Movie: Naruto to Mashin to Mitsu no Onegai Dattebayo!!',
 'Naruto: Shippuuden Movie 4 - The Lost Tower',
 'Boruto: Naruto the Movie',
 'Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono']